In [3]:
#import libraries
# import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# load FashionMNIST
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# CNN architecture
class CNN_FashionMNIST(nn.Module):
    def __init__(self):
        super(CNN_FashionMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 256)  # after 2 poolings (28→14→7)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# create model
model = CNN_FashionMNIST()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# training
for epoch in range(20):  # train longer for better accuracy
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/20], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {acc:.2f}%")

# testing
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Epoch [1/20], Loss: 0.5897, Accuracy: 79.53%
Epoch [2/20], Loss: 0.3541, Accuracy: 87.39%
Epoch [3/20], Loss: 0.3049, Accuracy: 88.99%
Epoch [4/20], Loss: 0.2753, Accuracy: 90.03%
Epoch [5/20], Loss: 0.2564, Accuracy: 90.61%
Epoch [6/20], Loss: 0.2409, Accuracy: 91.31%
Epoch [7/20], Loss: 0.2290, Accuracy: 91.68%
Epoch [8/20], Loss: 0.2143, Accuracy: 92.19%
Epoch [9/20], Loss: 0.2055, Accuracy: 92.38%
Epoch [10/20], Loss: 0.1952, Accuracy: 92.87%
Epoch [11/20], Loss: 0.1864, Accuracy: 93.22%
Epoch [12/20], Loss: 0.1781, Accuracy: 93.50%
Epoch [13/20], Loss: 0.1724, Accuracy: 93.69%
Epoch [14/20], Loss: 0.1647, Accuracy: 93.87%
Epoch [15/20], Loss: 0.1572, Accuracy: 94.22%
Epoch [16/20], Loss: 0.1498, Accuracy: 94.46%
Epoch [17/20], Loss: 0.1442, Accuracy: 94.59%
Epoch [18/20], Loss: 0.1391, Accuracy: 94.76%
Epoch [19/20], Loss: 0.1348, Accuracy: 95.02%
Epoch [20/20], Loss: 0.1276, Accuracy: 95.23%
Test Accuracy: 91.83%
